### Finding small molecule inhibitors for COVID-19 Disease Map (DM) pathways

Fetch the relevant pathway model from MMINERVA, and extract model elements

In [1]:
from indra.sources.minerva.minerva_client import *
from covid_19.disease_maps.id_mapping_minerva import *

# Some basic elements of the COVID-19 DM
config = get_config(default_map_name)
project_id = get_project_id_from_config(config)
models = get_models(project_id, default_map_name)

In [2]:
[m['name'] for m in models]

['overview',
 'E protein interactions',
 'Electron Transport Chain disruption',
 'Apoptosis pathway',
 'HMOX1 pathway',
 'Endoplasmatic Reticulum stress',
 'Interferon lambda pathway',
 'Pyrimidine deprivation',
 'Orf3a protein interactions',
 'Nsp14 and metabolism',
 'Coagulation pathway',
 'Nsp4 and Nsp6 protein interactions',
 'Orf10 Cul2 pathway',
 'Virus replication cycle',
 'TGFbeta signalling',
 'SARS-CoV-2 RTC and transcription',
 'PAMP signalling',
 'Interferon 1 pathway',
 'Nsp9 protein interactions',
 'Kynurenine synthesis pathway',
 'Renin-angiotensin pathway',
 'JNK pathway',
 'NLRP3 inflammasome activation']

In [3]:
model_ids = {m['name']: m['idObject'] for m in models if m['name'] != 'overview'}
model_elements = {model_name: get_model_elements(model_id, project_id, default_map_name)
                  for model_name, model_id in model_ids.items()}

Now construct the INDRA mappings (i.e., equivalents using INDRA-compatible grounding namespaces and IDs) for these elements

In [4]:
indra_mappings = {model_name: [indra_db_refs_from_minerva_refs(get_element_references(element))
                  for element in model_elements[model_name]] for model_name in model_ids}

INFO: [2021-04-29 17:01:13] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /Users/ben/.indra/bio_ontology/1.12/bio_ontology.pkl


In [5]:
indra_groundings = {model_name: set(get_prioritized_db_refs_key(db_refs)
                                    for db_refs in indra_mappings[model_name] if db_refs)
                    for model_name in model_ids}

In [6]:
indra_groundings

{'E protein interactions': {('CHEBI', 'CHEBI:137113'),
  ('CHEBI', 'CHEBI:15378'),
  ('CHEBI', 'CHEBI:29101'),
  ('CHEBI', 'CHEBI:29103'),
  ('GO', 'GO:0005634'),
  ('GO', 'GO:0005737'),
  ('HGNC', '100'),
  ('HGNC', '1103'),
  ('HGNC', '11588'),
  ('HGNC', '13575'),
  ('HGNC', '1599'),
  ('HGNC', '1780'),
  ('HGNC', '18669'),
  ('HGNC', '19420'),
  ('HGNC', '20237'),
  ('HGNC', '20505'),
  ('HGNC', '20510'),
  ('HGNC', '28881'),
  ('HGNC', '4025'),
  ('HGNC', '4026'),
  ('HGNC', '4027'),
  ('HGNC', '4028'),
  ('HGNC', '4029'),
  ('HGNC', '4030'),
  ('HGNC', '4034'),
  ('HGNC', '4760'),
  ('HGNC', '4766'),
  ('HGNC', '4781'),
  ('HGNC', '4793'),
  ('HGNC', '4794'),
  ('HGNC', '799'),
  ('HGNC', '804'),
  ('UP', 'E'),
  (None, None)},
 'Electron Transport Chain disruption': {('CHEBI', 'CHEBI:15377'),
  ('CHEBI', 'CHEBI:15379'),
  ('CHEBI', 'CHEBI:15422'),
  ('CHEBI', 'CHEBI:15846'),
  ('CHEBI', 'CHEBI:16234'),
  ('CHEBI', 'CHEBI:16238'),
  ('CHEBI', 'CHEBI:16240'),
  ('CHEBI', 'CHEBI:16

Filter down to all the human genes

In [7]:
human_genes = {model_name: {entry for entry in indra_groundings[model_name] if entry[0] == 'HGNC'}
               for model_name in model_ids}

In [8]:
for model_name, genes in human_genes.items():
    print(model_name, len(genes))

E protein interactions 26
Electron Transport Chain disruption 213
Apoptosis pathway 22
HMOX1 pathway 53
Endoplasmatic Reticulum stress 71
Interferon lambda pathway 50
Pyrimidine deprivation 26
Orf3a protein interactions 46
Nsp14 and metabolism 70
Coagulation pathway 58
Nsp4 and Nsp6 protein interactions 40
Orf10 Cul2 pathway 9
Virus replication cycle 6
TGFbeta signalling 23
SARS-CoV-2 RTC and transcription 0
PAMP signalling 45
Interferon 1 pathway 45
Nsp9 protein interactions 90
Kynurenine synthesis pathway 39
Renin-angiotensin pathway 21
JNK pathway 15
NLRP3 inflammasome activation 15


We can now turn to the INDRA DB and find Inhibition statements targeting each of these human genes in the pathway of interest. We filter regulators to ones grounded to small molecule name spaces. We also filter out statements that have been curated as incorrect.

In [9]:
from indra.config import get_config
from indra.sources import indra_db_rest
from indra.tools import assemble_corpus as ac
try:
    from indra_db import get_db
    from indra_db.client.principal.curation import get_curations
    curs = get_curations(get_db('primary'))
except Exception:
    curs = []
print(get_config('INDRA_DB_REST_URL'))

https://db.indra.bio/


In [10]:
def get_stmts_by_object(stmts):
    from indra.statements import Inhibition
    stmts_by_object = defaultdict(list)
    for stmt in stmts:
        if isinstance(stmt, Inhibition):
            obj_ns, obj_id = stmt.obj.get_grounding()
            if not obj_ns:
                continue
            stmts_by_object[(obj_ns, obj_id)].append(stmt)
    return dict(stmts_by_object)

def load_dgi():
    from indra.statements import Inhibition
    from indra.sources import dgi
    dp = dgi.process_version()
    return get_stmts_by_object(dp.statements)

def load_crog():
    from indra.sources import crog
    cp = crog.process_from_web()
    return get_stmts_by_object(cp.statements)

In [11]:
import os
import pickle
USE_CACHE = False

if USE_CACHE and os.path.exists('inhibitors.pkl'):
    with open('inhibitors.pkl', 'rb') as fh:
        inhibitors = pickle.load(fh)
else:
    inhibitors = {}
    crog_stmts = load_crog()
    dgi_stmts = load_dgi()
    for model, entries in human_genes.items():
        for db_ns, db_id in entries:
            if (db_ns, db_id) in inhibitors:
                continue
            ip = indra_db_rest.get_statements(object=f'{db_id}@{db_ns}', stmt_type='Inhibition',
                                              ev_limit=200)
            stmts = [s for s in ip.statements if set(s.subj.db_refs) & {'CHEBI', 'PUBCHEM', 'CAS', 'CHEMBL', 'DRUGBANK'}]
            stmts += crog_stmts.get((db_ns, db_id), [])
            stmts += dgi_stmts.get((db_ns, db_id), [])
            stmts = ac.run_preassembly(stmts, run_refinement=False)
            stmts = ac.filter_by_curation(stmts, curations=curs)
            inhibitors[(db_ns, db_id)] = stmts
    with open('inhibitors.pkl', 'wb') as fh:
        pickle.dump(inhibitors, fh)

        
from indra.databases.identifiers import ensure_chebi_prefix
from indra.ontology.standardize import standardize_agent_name
from indra.preassembler import Preassembler
for target, stmts in inhibitors.items():
    use_stmts = []
    for stmt in stmts:
        stmt.evidence = [ev for ev in stmt.evidence if ev.source_api != 'medscan']
        if not stmt.evidence:
            continue
        for agent in stmt.real_agent_list():
            if 'CHEBI' in agent.db_refs:
                agent.db_refs['CHEBI'] = ensure_chebi_prefix(agent.db_refs['CHEBI'])
                standardize_agent_name(agent, standardize_refs=True)
        use_stmts.append(stmt)
    pa = Preassembler(None)
    stmts = pa.combine_duplicate_stmts(use_stmts)
    inhibitors[target] = stmts

INFO: [2021-04-29 17:01:39] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=4027@HGNC&offset=0&type=Inhibition
INFO: [2021-04-29 17:01:39] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-04-29 17:01:40] indra.tools.assemble_corpus - Combining duplicates on 0 statements...
INFO: [2021-04-29 17:01:40] indra.tools.assemble_corpus - 0 unique statements
INFO: [2021-04-29 17:01:40] indra.tools.assemble_corpus - Filtering 0 statements with any incorrect curations...
INFO: [2021-04-29 17:01:40] indra.tools.assemble_corpus - 0 statements after filter...
INFO: [2021-04-29 17:01:40] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=4030@HGNC&offset=0&type=Inhibition
INFO: [2021-04-29 17:01:40] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-04-29 17:01:41] indra.tools.assemb

INFO: [2021-04-29 17:01:57] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=13575@HGNC&offset=0&type=Inhibition
INFO: [2021-04-29 17:01:57] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-04-29 17:02:00] indra.tools.assemble_corpus - Combining duplicates on 62 statements...
INFO: [2021-04-29 17:02:00] indra.tools.assemble_corpus - 58 unique statements
INFO: [2021-04-29 17:02:00] indra.tools.assemble_corpus - Filtering 58 statements with any incorrect curations...
INFO: [2021-04-29 17:02:00] indra.tools.assemble_corpus - 58 statements after filter...
INFO: [2021-04-29 17:02:00] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=4794@HGNC&offset=0&type=Inhibition
INFO: [2021-04-29 17:02:00] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-04-29 17:02:01] indra.tools.a

INFO: [2021-04-29 17:02:12] indra.tools.assemble_corpus - Combining duplicates on 51 statements...
INFO: [2021-04-29 17:02:12] indra.tools.assemble_corpus - 45 unique statements
INFO: [2021-04-29 17:02:12] indra.tools.assemble_corpus - Filtering 45 statements with any incorrect curations...
INFO: [2021-04-29 17:02:12] indra.tools.assemble_corpus - 45 statements after filter...
INFO: [2021-04-29 17:02:12] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=19420@HGNC&offset=0&type=Inhibition
INFO: [2021-04-29 17:02:12] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-04-29 17:02:13] indra.tools.assemble_corpus - Combining duplicates on 1 statements...
INFO: [2021-04-29 17:02:13] indra.tools.assemble_corpus - 1 unique statements
INFO: [2021-04-29 17:02:13] indra.tools.assemble_corpus - Filtering 1 statements with any incorrect curations...
INFO: [2021-04-29 17:02:13] indra.tools.a

INFO: [2021-04-29 17:02:23] indra.tools.assemble_corpus - 5 statements after filter...
INFO: [2021-04-29 17:02:23] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=843@HGNC&offset=0&type=Inhibition
INFO: [2021-04-29 17:02:23] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-04-29 17:02:25] indra.tools.assemble_corpus - Combining duplicates on 0 statements...
INFO: [2021-04-29 17:02:25] indra.tools.assemble_corpus - 0 unique statements
INFO: [2021-04-29 17:02:25] indra.tools.assemble_corpus - Filtering 0 statements with any incorrect curations...
INFO: [2021-04-29 17:02:25] indra.tools.assemble_corpus - 0 statements after filter...
INFO: [2021-04-29 17:02:25] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=9179@HGNC&offset=0&type=Inhibition
INFO: [2021-04-29 17:02:25] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'bes

INFO: [2021-04-29 17:02:38] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-04-29 17:02:39] indra.tools.assemble_corpus - Combining duplicates on 0 statements...
INFO: [2021-04-29 17:02:39] indra.tools.assemble_corpus - 0 unique statements
INFO: [2021-04-29 17:02:39] indra.tools.assemble_corpus - Filtering 0 statements with any incorrect curations...
INFO: [2021-04-29 17:02:39] indra.tools.assemble_corpus - 0 statements after filter...
INFO: [2021-04-29 17:02:39] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=24380@HGNC&offset=0&type=Inhibition
INFO: [2021-04-29 17:02:39] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-04-29 17:02:39] indra.tools.assemble_corpus - Combining duplicates on 1 statements...
INFO: [2021-04-29 17:02:39] indra.tools.assemble_corpus - 1 unique statements
INFO: [2021-04-29 17:02:3

...

INFO: [2021-04-29 17:30:13] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-04-29 17:30:14] indra.tools.assemble_corpus - Combining duplicates on 1 statements...
INFO: [2021-04-29 17:30:14] indra.tools.assemble_corpus - 1 unique statements
INFO: [2021-04-29 17:30:14] indra.tools.assemble_corpus - Filtering 1 statements with any incorrect curations...
INFO: [2021-04-29 17:30:14] indra.tools.assemble_corpus - 1 statements after filter...
INFO: [2021-04-29 17:30:14] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=16902@HGNC&offset=0&type=Inhibition
INFO: [2021-04-29 17:30:14] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-04-29 17:30:15] indra.tools.assemble_corpus - Combining duplicates on 9 statements...
INFO: [2021-04-29 17:30:15] indra.tools.assemble_corpus - 9 unique statements
INFO: [2021-04-29 17:30:1

INFO: [2021-04-29 17:31:03] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=19119@HGNC&offset=0&type=Inhibition
INFO: [2021-04-29 17:31:03] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-04-29 17:31:04] indra.tools.assemble_corpus - Combining duplicates on 16 statements...
INFO: [2021-04-29 17:31:04] indra.tools.assemble_corpus - 15 unique statements
INFO: [2021-04-29 17:31:04] indra.tools.assemble_corpus - Filtering 15 statements with any incorrect curations...
INFO: [2021-04-29 17:31:04] indra.tools.assemble_corpus - 15 statements after filter...
INFO: [2021-04-29 17:31:04] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=19288@HGNC&offset=0&type=Inhibition
INFO: [2021-04-29 17:31:04] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-04-29 17:31:05] indra.tools.

INFO: [2021-04-29 17:31:31] indra.tools.assemble_corpus - 4 unique statements
INFO: [2021-04-29 17:31:31] indra.tools.assemble_corpus - Filtering 4 statements with any incorrect curations...
INFO: [2021-04-29 17:31:31] indra.tools.assemble_corpus - 4 statements after filter...
INFO: [2021-04-29 17:31:31] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=348@HGNC&offset=0&type=Inhibition
INFO: [2021-04-29 17:31:31] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-04-29 17:31:36] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=348@HGNC&offset=500&type=Inhibition
INFO: [2021-04-29 17:31:36] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-04-29 17:31:38] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=348@HGNC&offset=1000&ty

INFO: [2021-04-29 17:31:53] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=7154@HGNC&offset=0&type=Inhibition
INFO: [2021-04-29 17:31:53] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-04-29 17:31:56] indra.tools.assemble_corpus - Combining duplicates on 158 statements...
INFO: [2021-04-29 17:31:56] indra.tools.assemble_corpus - 154 unique statements
INFO: [2021-04-29 17:31:56] indra.tools.assemble_corpus - Filtering 154 statements with any incorrect curations...
INFO: [2021-04-29 17:31:56] indra.tools.assemble_corpus - 154 statements after filter...
INFO: [2021-04-29 17:31:56] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=11793@HGNC&offset=0&type=Inhibition
INFO: [2021-04-29 17:31:56] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-04-29 17:31:56] indra.too

INFO: [2021-04-29 17:32:17] indra.tools.assemble_corpus - Combining duplicates on 49 statements...
INFO: [2021-04-29 17:32:17] indra.tools.assemble_corpus - 47 unique statements
INFO: [2021-04-29 17:32:17] indra.tools.assemble_corpus - Filtering 47 statements with any incorrect curations...
INFO: [2021-04-29 17:32:17] indra.tools.assemble_corpus - 47 statements after filter...
INFO: [2021-04-29 17:32:17] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?object=6206@HGNC&offset=0&type=Inhibition
INFO: [2021-04-29 17:32:17] indra.sources.indra_db_rest.util - params: {'ev_limit': 200, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-04-29 17:32:18] indra.tools.assemble_corpus - Combining duplicates on 14 statements...
INFO: [2021-04-29 17:32:18] indra.tools.assemble_corpus - 14 unique statements
INFO: [2021-04-29 17:32:18] indra.tools.assemble_corpus - Filtering 14 statements with any incorrect curations...
INFO: [2021-04-29 17:32:18] indra.tools

We can now check how many targets there are small molecule inhibitors for

In [12]:
has_inhibitors = len([gene for gene, inhs in inhibitors.items() if inhs])
print('We found inhibitors for %d of %d targets' % (has_inhibitors, len(inhibitors)))

We found inhibitors for 641 of 805 targets


In [13]:
readers = {'reach', 'sparser', 'trips', 'isi', 'rlimsp', 'eidos', 'medscan'}
def get_top_inhibitors(inhs):
    def get_sources(stmt):
        return {ev.source_api for ev in stmt.evidence}
    has_db_support = sorted([stmt for stmt in inhs if
                             get_sources(stmt) - readers],
                            key=lambda x: len(x.evidence),
                            reverse=True)
    no_db_support = sorted([stmt for stmt in inhs if
                           get_sources(stmt) <= readers],
                           key=lambda x: len(x.evidence),
                           reverse=True)
    return {'db_support': has_db_support[:5], 'no_db_support': no_db_support[:5]}
    

In [14]:
top_inhibitors = {target: get_top_inhibitors(inhs) for target, inhs in inhibitors.items()}

In [15]:
top_inhibitors

{('HGNC', '4027'): {'db_support': [], 'no_db_support': []},
 ('HGNC', '4030'): {'db_support': [], 'no_db_support': []},
 ('HGNC',
  '1780'): {'db_support': [Inhibition(alvocidib(), CDK9()),
   Inhibition(2-[(2S)-1-[3-ethyl-7-[(1-oxido-3-pyridin-1-iumyl)methylamino]-5-pyrazolo[1,5-a]pyrimidinyl]-2-piperidinyl]ethanol(), CDK9()),
   Inhibition(seliciclib(), CDK9()),
   Inhibition(N-(5-\{[(5-tert-butyl-1,3-oxazol-2-yl)methyl]sulfanyl\}-1,3-thiazol-2-yl)piperidine-4-carboxamide(), CDK9()),
   Inhibition(FLAVOPYRIDOL(), CDK9())], 'no_db_support': [Inhibition(wogonin(), CDK9()),
   Inhibition(iron atom(), CDK9()),
   Inhibition(3',5'-di-O-methyltricetin(), CDK9()),
   Inhibition(7H-purine(), CDK9()),
   Inhibition(ATP(), CDK9())]},
 ('HGNC',
  '4026'): {'db_support': [Inhibition(digitoxigenin(), FXYD2()),
   Inhibition(digitoxin(), FXYD2()),
   Inhibition(cyclothiazide(), FXYD2()),
   Inhibition(digoxin(), FXYD2()),
   Inhibition(DESGLUCOUZARIN(), FXYD2())], 'no_db_support': [Inhibition(cycl

In [16]:
import pandas
open_targets_url = ('https://storage.googleapis.com/open-targets-data-releases/'
 '21.02/input/annotation-files/tractability_buckets-2021-01-12.tsv')
df = pandas.read_csv(open_targets_url, sep='\t')
cols = [ 'Top_bucket_PROTAC',
 'Top_bucket_ab',
 'Top_bucket_othercl',
 'Top_bucket_sm',
]
target_scores = {}
for _, row in df.iterrows():
    vals = [row[c] for c in cols]
    target_scores[row['symbol']] = min(vals)

/Users/ben/.virtualenvs/py39/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (15,16,40,75,76,96) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
target_scores['IL6']

1

In [18]:
from collections import Counter
from indra.databases import identifiers
from indra.statements.agent import default_ns_order
from indra.databases import hgnc_client
ns_order = default_ns_order + ['DRUGBANK', 'PUBCHEM', 'CAS', 'CHEMBL']

def get_table_rows(target, stmts_by_support):
    target_symbol = hgnc_client.get_hgnc_name(target[1])
    target_id = '%s:%s' % ('hgnc', target[1])
    rows = []
    for support_type in ['db_support', 'no_db_support']:
        for stmt in stmts_by_support[support_type]:
            drug = stmt.subj
            drug_name = drug.name
            drug_ns, drug_id = drug.get_grounding(ns_order)
            if drug_ns:
                identifiers_ns = identifiers.get_identifiers_ns(drug_ns)
                drug_ns = identifiers_ns if identifiers_ns else drug_ns
                drug_grounding = '%s:%s' % (drug_ns, drug_id)
            else:
                drug_grounding = ''
            pmids = sorted({ev.pmid for ev in stmt.evidence if ev.pmid})[:10]
            pmids_str = ', '.join(pmids) if pmids else ''
            sources = Counter([ev.source_api for ev in stmt.evidence])
            sources_str = ', '.join('%s:%s' % (source_name, source_count)
                                    for source_name, source_count in sources.most_common())
            row = [target_symbol, target_id, drug_name, drug_grounding, support_type, sources_str, pmids_str]
            rows.append(row)
    return rows

def target_sort_key(target):
    target_symbol = hgnc_client.get_hgnc_name(target[1])
    target_score_key = target_scores.get(target_symbol, 10)
    ev_key = 0
    inhs = top_inhibitors[target]
    ev_key += sum([len(stmt.evidence) for stmt in inhs['db_support'] + inhs['no_db_support']])
    # We put - for ev_key to make it sort in the right direction
    return (target_score_key, -ev_key)

all_rows = []
for target, inh_stmts in sorted(top_inhibitors.items(), key=lambda x: target_sort_key(x[0])):
    all_rows += get_table_rows(target, inh_stmts)
with open('inhibitors.csv', 'w') as fh:
    writer = csv.writer(fh)
    writer.writerows(all_rows)

In [19]:
from indra.assemblers.html import HtmlAssembler
from indra.ontology.bio import bio_ontology
all_stmts = []
# Dump target-specific HTMLs
for gene_grounding, inhs_by_support in top_inhibitors.items():
    inhs = inhs_by_support['db_support'] + inhs_by_support['no_db_support']
    ha = HtmlAssembler(inhs, db_rest_url='https://db.indra.bio')
    ha.make_model()
    gene_name = bio_ontology.get_name(*gene_grounding)
    ha.save_model(f'{gene_name}.html')
    all_stmts += inhs
# Now dump for all inhibitors in a single file
ha = HtmlAssembler(all_stmts, db_rest_url='https://db.indra.bio')
ha.make_model()
ha.save_model(f'all_inhibitors.html')

INFO: [2021-04-29 17:33:52] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'CHEBI:95080', 'CHEBI:137113'}
INFO: [2021-04-29 17:33:52] indra.assemblers.html.assembler - Removing PUBCHEM from refs due to too many matches: {'439153', '928'}
INFO: [2021-04-29 17:33:53] indra.assemblers.html.assembler - Removing PUBCHEM from refs due to too many matches: {'439153', '928'}
INFO: [2021-04-29 17:33:54] indra.assemblers.html.assembler - Removing PUBCHEM from refs due to too many matches: {'4971', '53481561'}
INFO: [2021-04-29 17:33:54] indra.assemblers.html.assembler - Removing PUBCHEM from refs due to too many matches: {'6914273', '2952'}
INFO: [2021-04-29 17:33:55] indra.assemblers.html.assembler - Removing PUBCHEM from refs due to too many matches: {'9863776', '68810458'}
INFO: [2021-04-29 17:33:56] indra.assemblers.html.assembler - Removing PUBCHEM from refs due to too many matches: {'439542', '10909430'}
INFO: [2021-04-29 17:33:57] indra.assemblers.htm

In [20]:
# Now dump for all inhibitors in a single file
ha = HtmlAssembler(all_stmts, db_rest_url='https://db.indra.bio')
ha.make_model()
ha.save_model(f'all_inhibitors.html')

INFO: [2021-04-29 17:34:05] indra.assemblers.html.assembler - Removing CHEBI from refs due to too many matches: {'CHEBI:95080', 'CHEBI:137113'}
INFO: [2021-04-29 17:34:05] indra.assemblers.html.assembler - Removing CAS from refs due to too many matches: {'116314-67-1', '130929-57-6'}
INFO: [2021-04-29 17:34:05] indra.assemblers.html.assembler - Removing PUBCHEM from refs due to too many matches: {'4971', '53481561'}
INFO: [2021-04-29 17:34:06] indra.assemblers.html.assembler - Removing PUBCHEM from refs due to too many matches: {'6914273', '2952'}
INFO: [2021-04-29 17:34:06] indra.assemblers.html.assembler - Removing PUBCHEM from refs due to too many matches: {'9863776', '68810458'}
INFO: [2021-04-29 17:34:06] indra.assemblers.html.assembler - Removing PUBCHEM from refs due to too many matches: {'439542', '10909430'}
INFO: [2021-04-29 17:34:06] indra.assemblers.html.assembler - Removing PUBCHEM from refs due to too many matches: {'439153', '928'}
INFO: [2021-04-29 17:34:06] indra.assem